In [1]:
%pip install azure-ai-vision-imageanalysis

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential
from PIL import Image, ImageDraw, ImageFont
import gradio as gr

# Set the values of your computer vision endpoint and computer vision key
# as environment variables:
def request_vision(image_path):
    endpoint = "https://fimtrus-computervision2.cognitiveservices.azure.com/"
    key = "ee6d3d3de333480291b572fc7147b015"

    features = VisualFeatures.OBJECTS

    # Create an Image Analysis client
    client = ImageAnalysisClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key)
    )

    with open(image_path, 'rb') as image:
        image_data = image.read()

    # Get a caption for the image. This will be a synchronously (blocking) call.
    result = client._analyze_from_image_data(
        # language='ko',
        # smart_crops_aspect_ratios=[1.2,0.75,1.8],
        # image_url="https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png",
        image_data=image_data,
        visual_features=[VisualFeatures.OBJECTS], #VisualFeatures.CAPTION, VisualFeatures.READ],
        gender_neutral_caption=True,  # Optional (default is False)
    )

    return result

def change_image(image_path):

    if image_path is None:
        return None
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("NanumGothic.ttf", 30)
    except IOError:
        # Arial 폰트가 없으면 기본 폰트를 사용합니다.
        font = ImageFont.load_default()

    response = request_vision(image_path)

    if response and response.objects and response.objects.list:
        item_list = response.objects.list

        for item in item_list:
           box = item.get('boundingBox')
           x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
           # item = dict
           draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
           if item.tags:  
                for tag in item.tags:  
                    draw.text((x, y - 10), tag["name"], fill="red", font=font)

    return image

with gr.Blocks() as demo:

    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)

    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

# demo.launch()
change_image('/Users/fimtrus/Pictures/002.다이빙/501be83cf4a806200d4ab58d39ba71db_thumbnail.JPG')


KeyboardInterrupt: 